## 1번

In [ ]:
x_padding=[[0,0,0,0,0,0,0,0],
           [0,2,0,2,1,3,2,0],
           [0,0,2,0,2,2,2,0],
           [0,1,0,1,3,1,1,0],
           [0,0,0,1,1,1,0,0],
           [0,0,1,3,4,1,0,0],
           [0,0,1,0,0,5,2,0],
           [0,0,0,0,0,0,0,0]]

answer_1=[[4,0,4,3,5,2],
          [0,5,3,5,4,5],
          [1,1,4,4,3,3],
          [1,4,5,3,4,1],
          [1,1,3,10,4,1],
          [0,1,1,3,9,3]]

answer_2=[[2,2,3,2],
          [1,2,3,2],
          [0,3,4,0],
          [0,1,5,2]]

## 2번
1. Layer1의 Kernel size를 4로 늘리고, Layer3의 stride를 2로 늘려라
2. Layer3의 Output size를 구하여라
3. Cost와 accuracy가 기존 CNN모델에 비해 어떻게 변할 것인가?
4. 코딩하여 cost와 accuracy를 확인해보자!

## 정답

- Layer 1
    <br>
    Convolution Layer:(in_c=1, out_c=32, kernel_size=4, stride=1, padding=1)
    <br>
    MaxPool Layer: (kernel_size=2, stride=2)
    <br>
    <br>
- Layer 2
    <br>
    Convolution Layer:(in_c=32, out_c=64, kernel_size=3, stride=1, padding=1)
    <br>
    MaxPool Layer: (kernel_size=2, stride=2)
    <br>
    <br>
- **Layer 3**
    <br>
    Convolution Layer:(in_c=64, out_c=128, kernel_size=3, stride=2, padding=1)
    <br>
    MaxPool Layer: (kernel_size=2, stride=2)
    <br>
    <br>

- view
    <br>
    batch_size x [1,1,128] $\Rightarrow$ batch_size x[128]
    <br>
    <br>

- **Layer 4**
    <br>
    Fully Connected Layer:(in_c=1*1*128, out_c=625)
    <br>
    <br>
    
- **Layer 5**
    <br>
    Fully Connected Layer:(in_c=625, out_c=10)

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
import torch.nn as nn

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
#MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [5]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [6]:
# CNN Model
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=4, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2))
        #outputsize=13
        
        self.layer2 = nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2))
        
        #outputsize=6
    
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            torch.nn.MaxPool2d(2))

        #outputsize=1
        
        self.fc1 = torch.nn.Linear(1 * 1 * 128, 625)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [7]:
# instantiate CNN model
model = CNN().to(device)

value=(torch.Tensor(1,1,28,28)).to(device)
print( (model(value)).shape ) #임의의 value값을 넣어 test!

torch.Size([1, 10])


In [8]:
# define cost/loss & optimizer
criterion = nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
# train my model
total_batch = len(data_loader)
model.train()    # set the model to train mode (dropout=True)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 0.304485738
[Epoch:    2] cost = 0.106414095
[Epoch:    3] cost = 0.0813134015
[Epoch:    4] cost = 0.0647909269
[Epoch:    5] cost = 0.0548969656
[Epoch:    6] cost = 0.0465635657
[Epoch:    7] cost = 0.0384750813
[Epoch:    8] cost = 0.0341844857
[Epoch:    9] cost = 0.0297093503
[Epoch:   10] cost = 0.0275054481
[Epoch:   11] cost = 0.0228632726
[Epoch:   12] cost = 0.0208994225
[Epoch:   13] cost = 0.0162081141
[Epoch:   14] cost = 0.0175381638
[Epoch:   15] cost = 0.0165862199
Learning Finished!


In [10]:
# Test model and check accuracy
with torch.no_grad():
    model.eval()    # set the model to evaluation mode (dropout=False)

    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

C:\anaconda\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\anaconda\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.8935999870300293
